In [126]:
import os
import pathlib
import warnings
import glob as gl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy import log as ln
from sklearn.cluster import KMeans
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from matplotlib.axes._axes import _log as matplotlib_axes_logger
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn import metrics
from sklearn.decomposition import PCA
matplotlib_axes_logger.setLevel('ERROR')
warnings.filterwarnings('ignore')

In [127]:
plt.rc('text', usetex=False)
plt.rc('font', family='serif')
plt.rcParams.update({'font.size': 10})

In [128]:
data = pd.read_csv('/Users/alduck/Documents/PhDStuff_repo/PasiveTestingUsingMTplusFuzzTesting/ClusteringExploration/kmeans_k5_PCA_full.csv', index_col= 0)

In [129]:
data

,input_a,input_b,operand,operand_str,output,MR1_input_a,MR1_input_b,MR1_output,constant,MR2_input_a,...,MR3_checker_str,MR4_checker,MR4_checker_str,Component 1,Component 2,Component 3,Component 4,Component 5,KMeans,segment
0,3,0,1,sub,3.0,0.0,3.0,-3.0,5.0,15.0,...,No-violated,1.0,No-violated,-2.268467,-0.504003,0.845797,-2.795418,-0.325476,1,seconds
1,7,5,1,sub,2.0,5.0,7.0,-2.0,8.0,56.0,...,No-violated,1.0,No-violated,-0.743348,-2.101733,2.873795,1.560413,-0.615122,1,seconds
2,7,1,1,sub,6.0,1.0,7.0,-6.0,7.0,49.0,...,No-violated,1.0,No-violated,-0.741498,-0.766976,3.651138,-1.412954,-0.156625,1,seconds
3,4,0,1,sub,4.0,0.0,4.0,-4.0,1.0,4.0,...,No-violated,1.0,No-violated,-2.897382,1.695164,0.548983,-2.841739,-0.607392,1,seconds
4,1,4,1,sub,-3.0,4.0,1.0,3.0,2.0,2.0,...,No-violated,1.0,No-violated,-3.948147,-0.073950,-2.066120,-0.339249,0.639605,3,fourth
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,8,7,0,sum,15.0,7.0,8.0,15.0,1.0,8.0,...,Violated,0.0,Violated,2.204567,4.313280,-0.574161,1.737794,0.033270,0,first
196,1,9,0,sum,10.0,9.0,1.0,10.0,3.0,3.0,...,Violated,0.0,Violated,1.453773,0.860626,-3.783463,2.022811,-1.259818,0,first
197,3,1,0,sum,4.0,1.0,3.0,4.0,3.0,9.0,...,Violated,0.0,Violated,0.602191,0.859218,-1.719801,-2.785744,-0.093496,4,five
198,1,4,0,sum,5.0,4.0,1.0,5.0,5.0,5.0,...,Violated,0.0,Violated,1.179660,-0.802328,-2.901725,-1.054510,-0.631667,4,five


In [130]:
data_first = data.loc[data['segment']=='first']
data_second = data.loc[data['segment']=='seconds']
data_third = data.loc[data['segment']=='third']
data_fourth = data.loc[data['segment']=='fourth']
data_five = data.loc[data['segment']=='five']

In [131]:
data_first['MR1_checker_str'].value_counts()

No-violated    30
Name: MR1_checker_str, dtype: int64

In [132]:
data_first['MR2_checker_str'].value_counts()

No-violated    22
Violated        8
Name: MR2_checker_str, dtype: int64

In [133]:
data_first['MR3_checker_str'].value_counts()

Violated       22
No-violated     8
Name: MR3_checker_str, dtype: int64

In [134]:
data_first['MR4_checker_str'].value_counts()

Violated       22
No-violated     8
Name: MR4_checker_str, dtype: int64

In [135]:
if data_first['input_a'].dtype == str:
    print('adfa')

In [136]:
data_first.keys()

Index(['input_a', 'input_b', 'operand', 'operand_str', 'output', 'MR1_input_a',
       'MR1_input_b', 'MR1_output', 'constant', 'MR2_input_a', 'MR2_input_b',
       'MR2_output', 'MR3_input_a', 'MR3_input_b', 'MR3_output', 'MR4_input_a',
       'MR4_input_b', 'MR4_output', 'MR1_checker', 'MR1_checker_str',
       'MR2_checker', 'MR2_checker_str', 'MR3_checker', 'MR3_checker_str',
       'MR4_checker', 'MR4_checker_str', 'Component 1', 'Component 2',
       'Component 3', 'Component 4', 'Component 5', 'KMeans', 'segment'],
      dtype='object')

In [137]:
def encoder(df):
    data = df.copy()
    listKey = list(data.keys())


    for key in listKey:
        currentlistKey = list(data.keys())
        if data[key].dtype != object:
            try:
                indexAux = currentlistKey.index(key)
                data.insert(indexAux + 1,key + '_str', 0)

                for index, row in data.iterrows():
                    data.at[index, key + '_str'] = key + '==' + str(int(row[key])) 
                    
            except:
                continue
    return data
    

In [138]:
data_enc = encoder(data_first)
data_enc_2 = encoder(data_second)
data_enc_3 = encoder(data_third)
data_enc_4 = encoder(data_fourth)
data_enc_5 = encoder(data_five)

In [139]:
def encode_str(data_enc):
    listKey = list(data_enc.keys())
    data_str = data_enc.copy()

    for key in listKey:
        if data_enc[key].dtype != object:
            data_str.drop(key, inplace=True, axis=1)

        try:
            if key.find('Component') != -1:
                data_str.drop(key, inplace=True, axis=1)
        
        except:
            continue
    
    return data_str

In [140]:
data_enc = encode_str(data_enc)
data_enc_2 = encode_str(data_enc_2)
data_enc_3 = encode_str(data_enc_3)
data_enc_4 = encode_str(data_enc_4)
data_enc_5 = encode_str(data_enc_5)

In [141]:
data_enc.to_csv('firstClass_K5.csv')
data_enc_2.to_csv('secondClass_K5.csv')
data_enc_3.to_csv('thirdClass_K5.csv')
data_enc_4.to_csv('fourthClass_K5.csv')
data_enc_5.to_csv('fifthClass_K5.csv')


In [142]:
data_enc_2

,input_a_str,input_b_str,operand_str,output_str,MR1_input_a_str,MR1_input_b_str,MR1_output_str,constant_str,MR2_input_a_str,MR2_input_b_str,...,MR3_output_str,MR4_input_a_str,MR4_input_b_str,MR4_output_str,MR1_checker_str,MR2_checker_str,MR3_checker_str,MR4_checker_str,KMeans_str,segment
0,input_a==3,input_b==0,sub,output==3,MR1_input_a==0,MR1_input_b==3,MR1_output==-3,constant==5,MR2_input_a==15,MR2_input_b==0,...,MR3_output==3,MR4_input_a==-2,MR4_input_b==-5,MR4_output==3,Violated,No-violated,No-violated,No-violated,KMeans==1,seconds
1,input_a==7,input_b==5,sub,output==2,MR1_input_a==5,MR1_input_b==7,MR1_output==-2,constant==8,MR2_input_a==56,MR2_input_b==40,...,MR3_output==2,MR4_input_a==-1,MR4_input_b==-3,MR4_output==2,Violated,No-violated,No-violated,No-violated,KMeans==1,seconds
2,input_a==7,input_b==1,sub,output==6,MR1_input_a==1,MR1_input_b==7,MR1_output==-6,constant==7,MR2_input_a==49,MR2_input_b==7,...,MR3_output==6,MR4_input_a==0,MR4_input_b==-6,MR4_output==6,Violated,No-violated,No-violated,No-violated,KMeans==1,seconds
3,input_a==4,input_b==0,sub,output==4,MR1_input_a==0,MR1_input_b==4,MR1_output==-4,constant==1,MR2_input_a==4,MR2_input_b==0,...,MR3_output==4,MR4_input_a==3,MR4_input_b==-1,MR4_output==4,Violated,No-violated,No-violated,No-violated,KMeans==1,seconds
5,input_a==2,input_b==1,sub,output==1,MR1_input_a==1,MR1_input_b==2,MR1_output==-1,constant==3,MR2_input_a==6,MR2_input_b==3,...,MR3_output==1,MR4_input_a==-1,MR4_input_b==-2,MR4_output==1,Violated,No-violated,No-violated,No-violated,KMeans==1,seconds
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,input_a==6,input_b==3,sub,output==3,MR1_input_a==3,MR1_input_b==6,MR1_output==-3,constant==5,MR2_input_a==30,MR2_input_b==15,...,MR3_output==3,MR4_input_a==1,MR4_input_b==-2,MR4_output==3,Violated,No-violated,No-violated,No-violated,KMeans==1,seconds
94,input_a==4,input_b==1,sub,output==3,MR1_input_a==1,MR1_input_b==4,MR1_output==-3,constant==1,MR2_input_a==4,MR2_input_b==1,...,MR3_output==3,MR4_input_a==3,MR4_input_b==0,MR4_output==3,Violated,No-violated,No-violated,No-violated,KMeans==1,seconds
95,input_a==8,input_b==7,sub,output==1,MR1_input_a==7,MR1_input_b==8,MR1_output==-1,constant==1,MR2_input_a==8,MR2_input_b==7,...,MR3_output==1,MR4_input_a==7,MR4_input_b==6,MR4_output==1,Violated,No-violated,No-violated,No-violated,KMeans==1,seconds
97,input_a==3,input_b==1,sub,output==2,MR1_input_a==1,MR1_input_b==3,MR1_output==-2,constant==3,MR2_input_a==9,MR2_input_b==3,...,MR3_output==2,MR4_input_a==0,MR4_input_b==-2,MR4_output==2,Violated,No-violated,No-violated,No-violated,KMeans==1,seconds


In [143]:
data_enc

,input_a_str,input_b_str,operand_str,output_str,MR1_input_a_str,MR1_input_b_str,MR1_output_str,constant_str,MR2_input_a_str,MR2_input_b_str,...,MR3_output_str,MR4_input_a_str,MR4_input_b_str,MR4_output_str,MR1_checker_str,MR2_checker_str,MR3_checker_str,MR4_checker_str,KMeans_str,segment
109,input_a==8,input_b==6,sum,output==14,MR1_input_a==6,MR1_input_b==8,MR1_output==14,constant==0,MR2_input_a==0,MR2_input_b==0,...,MR3_output==14,MR4_input_a==8,MR4_input_b==6,MR4_output==14,No-violated,Violated,No-violated,No-violated,KMeans==0,first
110,input_a==4,input_b==8,sum,output==12,MR1_input_a==8,MR1_input_b==4,MR1_output==12,constant==3,MR2_input_a==12,MR2_input_b==24,...,MR3_output==18,MR4_input_a==1,MR4_input_b==5,MR4_output==6,No-violated,No-violated,Violated,Violated,KMeans==0,first
111,input_a==7,input_b==8,sum,output==15,MR1_input_a==8,MR1_input_b==7,MR1_output==15,constant==0,MR2_input_a==0,MR2_input_b==0,...,MR3_output==15,MR4_input_a==7,MR4_input_b==8,MR4_output==15,No-violated,Violated,No-violated,No-violated,KMeans==0,first
112,input_a==9,input_b==7,sum,output==16,MR1_input_a==7,MR1_input_b==9,MR1_output==16,constant==3,MR2_input_a==27,MR2_input_b==21,...,MR3_output==22,MR4_input_a==6,MR4_input_b==4,MR4_output==10,No-violated,No-violated,Violated,Violated,KMeans==0,first
117,input_a==3,input_b==5,sum,output==8,MR1_input_a==5,MR1_input_b==3,MR1_output==8,constant==2,MR2_input_a==6,MR2_input_b==10,...,MR3_output==12,MR4_input_a==1,MR4_input_b==3,MR4_output==4,No-violated,No-violated,Violated,Violated,KMeans==0,first
118,input_a==9,input_b==5,sum,output==14,MR1_input_a==5,MR1_input_b==9,MR1_output==14,constant==0,MR2_input_a==0,MR2_input_b==0,...,MR3_output==14,MR4_input_a==9,MR4_input_b==5,MR4_output==14,No-violated,Violated,No-violated,No-violated,KMeans==0,first
122,input_a==4,input_b==9,sum,output==13,MR1_input_a==9,MR1_input_b==4,MR1_output==13,constant==2,MR2_input_a==8,MR2_input_b==18,...,MR3_output==17,MR4_input_a==2,MR4_input_b==7,MR4_output==9,No-violated,No-violated,Violated,Violated,KMeans==0,first
131,input_a==3,input_b==9,sum,output==12,MR1_input_a==9,MR1_input_b==3,MR1_output==12,constant==1,MR2_input_a==3,MR2_input_b==9,...,MR3_output==14,MR4_input_a==2,MR4_input_b==8,MR4_output==10,No-violated,No-violated,Violated,Violated,KMeans==0,first
132,input_a==6,input_b==8,sum,output==14,MR1_input_a==8,MR1_input_b==6,MR1_output==14,constant==4,MR2_input_a==24,MR2_input_b==32,...,MR3_output==22,MR4_input_a==2,MR4_input_b==4,MR4_output==6,No-violated,No-violated,Violated,Violated,KMeans==0,first
133,input_a==9,input_b==5,sum,output==14,MR1_input_a==5,MR1_input_b==9,MR1_output==14,constant==0,MR2_input_a==0,MR2_input_b==0,...,MR3_output==14,MR4_input_a==9,MR4_input_b==5,MR4_output==14,No-violated,Violated,No-violated,No-violated,KMeans==0,first
